In [12]:
using Pkg

#Pkg.add("Optim")
#Pkg.add("PyPlot")
#Pkg.add("LinearAlgebra")

using Optim
using PyPlot
using LinearAlgebra

In [52]:
function market_shares(p,alpha)
    return exp.(-alpha.*p) / (1 + sum( exp.(-alpha.*p) ) )
end


function mco_profits(p,phi,mu,alpha)
    return market_shares(p,alpha) .* ( p - mu .* phi )
end


function price_profits(phi,mu,alpha)
    diff =  5.
    p = [1.,1.]
    profits = [0.,0.]
    
    maxiter = 10
    while (maxiter >=0) & (diff > 10e-7)
        #set up objective
        obj1(p0) = -1*mco_profits( [p0[1], p[2]], phi,mu,alpha)[1]
        res1 = optimize(obj1, [p[1]] , BFGS())
        
        obj2(p0) = -1*mco_profits( [p[1], p0[1]] ,phi,mu,alpha)[2]
        res2 = optimize(obj2,[p[2]], BFGS())
        
        #update loop variables
        diff = max( abs( res1.minimizer[1] - p[1] ), abs( res2.minimizer[1] - p[2] ) )
        maxiter = maxiter-1
        
        #update result
        p = [res1.minimizer[1],res2.minimizer[1]]
        profits =  [-1*res1.minimum, -1*res2.minimum]        
    end
    return p, profits
end


#arbitrary outside option... passive beliefs
function nash_in_nash(phi, mu, alpha)
    p, profits = price_profits(phi,mu,alpha)
    s = market_shares(p,alpha)
    return  profits .* sum(mu .* s .* phi)
end

print( nash_in_nash( [3,3], [.5,.5] , 1 ) )

[0.0141307, 0.0141307]

In [53]:
function bargain_helper(phi, mu, alpha)
    """solve each firm 1s optimization holding theta 2 fixed"""
    
    obj1(phi0) = -1*nash_in_nash( [phi0[1], phi[2]] , mu, alpha)[1]
    res1 =  optimize(obj1, [phi[1]], BFGS())

    obj2(phi0) = -1*nash_in_nash( [phi[1], phi0[1]] , mu, alpha)[2]
    res2 =  optimize(obj2, [phi[2]], BFGS())
    
    return [ res1.minimizer[1], res2.minimizer[1] ]
end

println( bargain_helper([2.,2.],[.5,.5], 1) )

[0.748419, 0.748419]


In [69]:
function simult_bargain(mu, alpha)
    """solve the bargaining problems seperately and iterate for a solution"""       
    
    #loop variables
    diff =  5.
    phi = [1., 1.]

    maxiter = 10
    while (maxiter >=0) & (diff > 10e-7)
        #seems as though there is a contraction mapping here, need to think more about why
        phi_new = bargain_helper(phi, mu, alpha)
        
        #update loop variables
        diff = maximum( abs.(phi - phi_new) )
        maxiter = maxiter-1
        
        #update phi
        phi = phi_new
    end
    return phi
end
    
print(simult_bargain([.5,.5],.5))

[1.71653, 1.71653]

In [ ]:
function bargain_helper_seq(phi1, mu, alpha):
    """ solve firm 1s optimization holding theta 2 fixed """
    obj1(phi2) = -1*nash_in_nash( [phi1, phi2[1] ] , mu, alpha)[1]
    res =  optimize(obj1, [2.], BFGS())
    return res.minimizer[1]
end

function seq_obj(phi1, mu, alpha):
    """ theta1 is a guess for the first stage """
    
    phi2 = bargain_helper_seq(phi1, mu2, mu1, cost, wtp)
    
    return nash_in_nash([phi1, phi2], mu, alpha)
end

In [ ]:
def seq_bargain(theta1, mu1, mu2, cost, wtp):
    """solve the bargaining problems seperately,
    then solve for the solution"""   
    #only compute no deal 1x for initial
    result = minimize(seq_obj, theta1, args=(mu1, mu2, cost, wtp),
                      method='Nelder-Mead', options={'disp': False})
    
    theta1 = result.x
    theta2 = bargain_helper(theta1, theta1, mu2, mu1, cost, wtp)
    return theta1, theta2

print(seq_bargain(17,1,1,5,25))